In [1]:
import os
import io
import torch
from transformers import AutoModel, AutoTokenizer,T5Tokenizer, T5ForConditionalGeneration,AutoModelForCausalLM
from fastapi import FastAPI, Depends, Response
from fastapi.responses import StreamingResponse
from res_models import ChatVO
import httpx
import json
from tqdm import trange
import ast
from gensim.models import KeyedVectors,word2vec
import itertools
import jieba
from sklearn.metrics.pairwise import cosine_similarity
import xiangsi as xs
import requests
import pandas as pd
import logging
import asyncio
import csv
import nest_asyncio
nest_asyncio.apply()
from lightrag import LightRAG, QueryParam
from lightrag.llm.ollama import ollama_model_complete, ollama_embed
from lightrag.kg.shared_storage import initialize_pipeline_status
from lightrag.utils import EmbeddingFunc
from typing import Any, AsyncIterator
from lightrag.operate import chunking_by_token_size
from torch.nn.parallel import DataParallel
import re

### 全局变量加载

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # 默认使用0号显卡，避免Windows用户忘记修改该处
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
knowledge=pd.read_csv('../../../data/total.csv') #疾病表格

with open('../../data/med_sys.txt','r') as f:
    data_line = f.readlines()
data_line = [i[:-1] for i in data_line]
with open('../../data/med.txt','r') as f:
    data_med = f.readlines()
data_med = [i[:-1] for i in data_med] #数据集加载
WORKING_DIR = "../../lightragpag/selfLightRAG/workapces"
logging.basicConfig(format="%(levelname)s:%(message)s", level=logging.INFO)
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)

### 函数和类

##### 1 加载知识图谱

In [3]:
async def initialize_rag():
    rag = LightRAG(
            working_dir=WORKING_DIR,
            chunking_func=chunking_by_token_size,
            llm_model_func=ollama_model_complete,
            llm_model_name="deepseek-r1:32b8192",
            llm_model_max_async=4,
            llm_model_max_token_size=131072,
            llm_model_kwargs={"host": "http://localhost:11434", "options": {"num_ctx": 8192}},
            embedding_func=EmbeddingFunc(
                embedding_dim=1024,
                max_token_size=8192,
                func=lambda texts: ollama_embed(
                    texts, embed_model="bge-m3:latest8192", host="http://localhost:11434"
                ),
            ),
        )
    await rag.initialize_storages()
    await initialize_pipeline_status()
    return rag

##### 2 扁鹊/小金刚模型加载

In [4]:
#访问扁鹊2模型
async def bianque2(content):
    url = "http://127.0.0.1:9290/bianque2"  
    data = {"content": f"{content}"}  
    response = requests.post(url, json=data)
    response = response.text
    result_str = ""
    for line in response.splitlines():  
        if line.startswith('data:'):   
            json_str = line.split('data: ')[1]
            if "is_end" not in json_str:
                continue
            data = json.loads(json_str)  
            result_str += data['result']
    return result_str
#访问小金刚模型
async def chimpanzee(prompt,content):
    device = "cuda"
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": content},
    ]
    # print(messages)
    inputs = chimpanzee_tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
    # print(messages)
    model_outputs = chimpanzee_model.generate(
        inputs,
        max_new_tokens=1024,
        temperature=0.7,
        top_p=0.7,
        pad_token_id=chimpanzee_tokenizer.eos_token_id  #设置 `pad_token_id` 为 `eos_token_id`
    ).to(device)
    # 解码生成的文本
    output_token_ids = [
        model_outputs[i][len(inputs[i]):] for i in range(len(inputs))
    ]
    responses = chimpanzee_tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
    responses = responses.replace("\n\n","")
    return responses

#访问微调小金刚模型
async def chimpanzeestf(prompt,content):
    device = "cuda"
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": content},
    ]
    inputs = chimpanzeestf_tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)
    model_outputs = chimpanzeestf_model.generate(
        inputs,
        max_new_tokens=1024,
        temperature=0.7,
        top_p=0.7,
        pad_token_id=chimpanzee_tokenizer.eos_token_id  #设置 `pad_token_id` 为 `eos_token_id`
    ).to(device)
    # 解码生成的文本
    output_token_ids = [
        model_outputs[i][len(inputs[i]):] for i in range(len(inputs))
    ]
    responses = chimpanzeestf_tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
    responses = responses.replace("\n\n","")
    return responses

### 加载模型

In [5]:
chimpanzeestf_path = "../../../../1_testmodel/stf/output1010/MiniCPM3_4B_5epoch"
bianque_v2_model_name_or_path = "../../../1_testmodel/ori/BianQue-2"
chimpanzee_path = "../../../../1_testmodel/ori/MiniCPM3-4B"
chimpanzeestf_tokenizer = AutoTokenizer.from_pretrained(chimpanzeestf_path,trust_remote_code=True)
chimpanzeestf_model = AutoModelForCausalLM.from_pretrained(chimpanzeestf_path, torch_dtype=torch.bfloat16, device_map='cuda', trust_remote_code=True)
chimpanzee_tokenizer = AutoTokenizer.from_pretrained(chimpanzee_path,trust_remote_code=True)
chimpanzee_model = AutoModelForCausalLM.from_pretrained(chimpanzee_path, torch_dtype=torch.bfloat16, device_map='cuda', trust_remote_code=True)
rag = asyncio.run(initialize_rag()) #知识图谱检索内容
# if torch.cuda.device_count() > 1:
#     chimpanzeestf_model = DataParallel(chimpanzeestf_model)
#     chimpanzee_model = DataParallel(chimpanzee_model)
# chimpanzeestf_model.to('cuda')
# chimpanzee_model.to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO: Process 48681 Shared-Data created for Single Process
INFO:Load (11748, 1024) data
INFO:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': '../../lightragpag/selfLightRAG/workapces/vdb_entities.json'} 11748 data
INFO:Load (33115, 1024) data
INFO:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': '../../lightragpag/selfLightRAG/workapces/vdb_relationships.json'} 33115 data
INFO:Load (2281, 1024) data
INFO:Init {'embedding_dim': 1024, 'metric': 'cosine', 'storage_file': '../../lightragpag/selfLightRAG/workapces/vdb_chunks.json'} 2281 data
INFO: Process 48681 initialized updated flags for namespace: [full_docs]
INFO: Process 48681 ready to initialize storage namespace: [full_docs]
INFO: Process 48681 initialized updated flags for namespace: [text_chunks]
INFO: Process 48681 ready to initialize storage namespace: [text_chunks]
INFO: Process 48681 initialized updated flags for namespace: [entities]
INFO: Process 48681 initialized updated flags for namespace:

### 访问流程

##### 1 根据最终疾病获得补充信息

In [6]:
async def get_kg_all(diseasessymptoms):
    triple =[]
    for i in range(len(knowledge)):
        name =knowledge['疾病名称'][i]
        synopsis =knowledge['简介'][i]
        symptoms = knowledge['症状'][i]
        combine =knowledge['并发症'][i]
        jzks =knowledge['就诊科室'][i]
        zlfs =knowledge['治疗方式'][i]
        zlzq =knowledge['治疗周期'][i]
        zyl =knowledge['治愈率'][i]
        cyyp =knowledge['常用药品'][i]
        zlfy =knowledge['治疗费用'][i]
        cause =knowledge['病因'][i]
        yffs =knowledge['预防方式'][i]
        zlgs =knowledge['治疗概述'][i]
        ycsw =knowledge['宜吃食物'][i]
        jcsw =knowledge['忌吃食物'][i]
        tjsp =knowledge['推荐食谱'][i]
        tjyp =knowledge['推荐药品'][i]
        ybjb =knowledge['医保疾病'][i]
        hbbl =knowledge['患病比例'][i]
        ygrq =knowledge['易感人群'][i]
        crfs =knowledge['传染方式'][i]
        triple.append((name,synopsis,symptoms,combine,jzks,zlfs,zlzq,zyl,cyyp,zlfy,cause,yffs,zlgs,ycsw, jcsw,tjsp,tjyp,ybjb,hbbl,ygrq,crfs))
    final = ""
    for name,synopsis,symptoms,combine,jzks,zlfs,zlzq,zyl,cyyp,zlfy,cause,yffs,zlgs,ycsw,jcsw,tjsp,tjyp,ybjb,hbbl,ygrq,crfs in triple:
        if isinstance(name,str) and name[:-4] in diseasessymptoms:
            synopsis = synopsis.replace('\n','')
            cause = cause.replace('\n','')
            final = final + f'疾病名称:{name};简介:{synopsis};症状:{symptoms};并发症:{combine};就诊科室:{jzks};治疗方式:{zlfs};治疗周期:{zlzq};治愈率:{zyl};常用药品:{cyyp};治疗费用:{zlfy};病因:{cause};预防方式:{yffs};治疗概述:{zlgs};宜吃食物:{ycsw};忌吃食物:{jcsw};推荐食谱:{tjsp};推荐药品:{tjyp};医保疾病:{ybjb};患病比例:{hbbl};易感人群:{ygrq};传染方式:{crfs};'
    return final

##### 2 可能症状可能性疾病-以及没有疾病的状况

In [7]:
async def kg_sym(history):
    example = '{"症状":["恶心","头晕"],"疾病":["内耳问题","颈椎病","贫血"]}'
    entity_prompt = "请从患者和医生的对话中提取关键信息。请识别患者描述的所有症状，以及经过医生诊断可能患有的疾病。将症状和疾病分别列出，确保你的答案是准确无误的,且一定要按照输出示例以 JSON 格式进行输出，不需要添加格外内容。"
    entity_input = f"""下列为患者和医生的对话历史:{history}\n
                    输出示例：{example}"""
    entity_responses = await chimpanzee(entity_prompt,entity_input) #小金刚提取症状和可能性疾病
    # print(entity_responses)
    entity_responses = entity_responses.replace('“', '"').replace('”', '"').replace('，', ',').replace(']]', ']').replace('[[', '[').replace("'", '"').replace("‘", '"').replace("’", '"').replace("json", '').replace("```", '').replace("\\", '').replace('心脏"漏跳"', '心脏漏跳')
    # print(entity_responses)
    # entity_responses = re.sub(
    #     r'("[^"]+")("[^"]+")',  # 匹配两个紧邻的引号部分（如 "心脏"漏跳"）
    #     lambda m: f'{m.group(1)}\\""{m.group(2)}',  # 在中间插入转义符
    #     entity_responses
    # )
    # print(entity_responses)
    entity_responses = json.loads(entity_responses)
    symptoms = entity_responses["症状"]
    candidate_diseases = entity_responses["疾病"]
    return symptoms,candidate_diseases

##### 3 评估是否满足

In [8]:
async def evalu(symptoms, diseases, extract_context):
    symptoms_str = '，'.join(symptoms)
    example_des = '{“analysis”:..., “distribution”: {“动物皮肤病”: 0.27, “红斑”: 0.3, “皮炎”: 0.85}}'
    des_prompt = "你是一名专业医生，任务是根据提供的症状信息对病人进行诊断。您将得到：可能的疾病科室、一份候选疾病清单和一些疾病的基础知识，您的任务是为患者提供详细的诊断分析和候选疾病的可信度分布。您需要首先分析患者的病情，并思考患者可能患有哪些候选疾病。如输出示例，以 JSON 格式输出分析和候选疾病的诊断置信度分布，请一定要按照输出示例格式进行输出。需要输出每一种候选疾病的诊断置信度分布，每种疾病的诊断置信度均在0-1之间，不需要总计为1，例如A疾病为0.8，B疾病可以为0.5。绝不可能患有的疾病置信度可以为0。"
    des_input = f"""输出示例：{example_des}\n
                    患者症状： {symptoms}\n
                    候选疾病： {diseases}\n
                    疾病基础信息：{extract_context}"""
    des_output = await chimpanzee(des_prompt,des_input)#小金刚进行分析和置信度获取
    try:
        des_responses = json.loads(des_output)
        analysis = des_responses["analysis"] #分析
        distribution = des_responses["distribution"] #打分
    except json.JSONDecodeError:
    # 如果 des_output 不是有效的 JSON 字符串，则处理异常
    # 这里可以返回一个空字典或者打印错误信息
        # print(des_output)
        des_responses = {}
        analysis = "" #分析
        distribution = {} #打分
    sorted_items = sorted(distribution.items(), key=lambda item: item[1], reverse=True)
    sim_de = dict(sorted_items)
    if list(sim_de.values())[0]>=0.5:
        ifxunhuan = False
    else:
        ifxunhuan = True
    return ifxunhuan,distribution, analysis

##### 4 获得最后的结果

In [9]:
async def get_end(distribution, query, hl_keywords ,ll_keywords):
    sorted_items = sorted(distribution.items(), key=lambda item: item[1], reverse=True)
    sim_de = dict(sorted_items)
    dise = list(sim_de.keys()) #疾病列表
    if len(dise)>5:
        dise = dise[:5] #获得前5个可能的疾病
        sim_de = dict(sorted(sim_de.items(), key=lambda item: item[0])[:5])
    
    history = "\n".join(hist.split("\n")[2:])
    conversation_history=[{"role": "user", "content": history}]
    results = rag.query(1, query, hl_keywords, dise, param=QueryParam(mode="ll_medical",conversation_history=conversation_history), ll_keywords=ll_keywords)    
    return results 

##### 5 获得疾病和症状列表

In [10]:
async def get_ills_sym(hist,result_hist):
    # print(hist)
    symptoms,candidate_diseases = await kg_sym(result_hist) #症状1和疾病1列表
    history = "\n".join(hist.split("\n")[2:])
    conversation_history=[{"role": "user", "content": history}]
    # print(conversation_history)
    response = rag.query(0, hist, data_line, data_med, param=QueryParam(mode="ll_medical",conversation_history=conversation_history))
    # print(response)
    response = response.split('11111')
    ll_keywords = response[0].split(", ") #症状2列表
    hl_keywords = response[1].split(", ")
    ills = response[2].split(", ") #疾病2列表
    diseases = ills + candidate_diseases #症状列表
    symptoms = ll_keywords + symptoms #疾病列表
    diseases = list(set(diseases))
    symptoms = list(set(symptoms))
    # print(diseases)
    # print(symptoms)
    return symptoms,diseases,hl_keywords

##### 6 获得疾病对应症状的补充信息

In [11]:
async def get_illsym(diseases):
    results = []
    data_line_n = [element.split(" ")[0] for element in data_line]
    for oo in diseases:
        result = [data_line[i] for i,element in enumerate(data_line_n) if oo==element]
        results = results + result
    else:
        results.append(oo + "症状:暂时没有症状信息。")
    result = []
    for oo in results:
        if oo not in result:
            result.append(oo)
    
    extract_context = "\n".join(result)
    return extract_context

##### 7 评估

In [14]:
with open('../data/CoD_data_kg516.json','r') as f:
    data_eval = f.readlines()
    data_eval = [json.loads(i) for i in data_eval]

FileNotFoundError: [Errno 2] No such file or directory: '../data/CoD_data_kg516.json'

In [ ]:
data_eval[0]

In [ ]:
for i_index, disease_data in enumerate(data_eval):
    if i_index>147:
        data_recoder = []
        data_recoder.append(disease_data['dies']) #目标疾病
        data_recoder.append(disease_data['competion']) #input输入
        input_text = "请依据医生患者的对话信息尽量给出一些大致的结果,建议的检查或者健康的生活及饮食习惯。\n对话信息:" + disease_data['competion'] + "医生："
        response = await bianque2(content=input_text) #bianque2 得出结论
        response = response.replace('医生：','').replace('患者：','')
        data_recoder.append(response)
        hist = "\n你是一名三甲医院的医生，请根据你和用户之间的对话并结合提供数据库，向用户提供诊断意见，诊断意见包括：可能性的疾病、疾病概述以及宜吃和忌吃食物等生活习惯。\n医患对话对话如下：\n" + disease_data['competion'] #医患对话历史（不包含医生的结论）
        result_hist = hist+ "医生：" + response
        # print(response)
        symptoms,diseases,hl_keywords = await get_ills_sym(hist,result_hist) #症状列表,疾病列表,查询关键词
        data_recoder.append(diseases)
        data_recoder.append(symptoms)
        extract_context = await get_illsym(diseases) #根据疾病补充症状信息
        data_recoder.append(extract_context)
        ifxunhuan,distribution,analysis = await evalu(symptoms, diseases, extract_context) #判断置信度是否大于0.5，False为是
        data_recoder.append(ifxunhuan)
        data_recoder.append(distribution)
        data_recoder.append(analysis)
        if ifxunhuan:
            hist_ask = disease_data['competion'] + "医生："
            re_or_in = f"""假设你是一名经验丰富的智能家庭助手，你的目标是根据患者和医生的对话信息，通过一系列有目的的提问来收集必要的信息，包括但不限于了解患者的症状、病史、生活方式和其他相关因素。不需要给出任何的意见，只需要进行提问，且每次仅可以提出1-2个问题，不可以提问重复的问题。下面为患者和医生的对话信息，请继续发起提问。\n\n对话信息:"""
            response = await chimpanzeestf(re_or_in,hist_ask) #继续提问
            data_recoder.append(response)
        else:
            response = await get_end(distribution, hist, hl_keywords, symptoms)
            response = response.replace('医生：','').replace('患者：','')
            data_recoder.append(response)

        # 追加写入文件
        with open('eval_answer_cod_kg.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(data_recoder)
        print(i_index)

In [15]:
data_eval = []
with open('../../data/eval_answer_cod_kg_extend_1.json','r') as f:
    data_eval_l = f.readlines()
for i in data_eval_l:
    # print(i)
    a = json.loads(i)
    data_eval.append(a)

In [20]:
for i_index, disease_data in enumerate(data_eval):
    if i_index>19:
        data_recoder = []
        data_recoder.append(disease_data['dies']) #目标疾病
        data_recoder.append(disease_data['competion']) #input输入
        input_text = "请依据医生患者的对话信息尽量给出一些大致的结果,建议的检查或者健康的生活及饮食习惯。\n对话信息:" + disease_data['competion'] + "医生："
        response = await bianque2(content=input_text) #bianque2 得出结论
        response = response.replace('医生：','').replace('患者：','')
        # data_recoder.append(response)
        hist = "\n你是一名三甲医院的医生，请根据你和用户之间的对话并结合提供数据库，向用户提供诊断意见，诊断意见包括：可能性的疾病、疾病概述以及宜吃和忌吃食物等生活习惯。\n医患对话对话如下：\n" + disease_data['competion'] #医患对话历史（不包含医生的结论）
        result_hist = hist+ "医生：" + response
        # print(response)
        symptoms,diseases,hl_keywords = await get_ills_sym(hist,result_hist) #症状列表,疾病列表,查询关键词
        # data_recoder.append(diseases)
        # data_recoder.append(symptoms)
        extract_context = await get_illsym(diseases) #根据疾病补充症状信息
        # data_recoder.append(extract_context)
        ifxunhuan,distribution,analysis = await evalu(symptoms, diseases, extract_context) #判断置信度是否大于0.5，False为是
        # data_recoder.append(ifxunhuan)
        data_recoder.append(distribution)
        # data_recoder.append(analysis)
        if ifxunhuan:
            hist_ask = disease_data['competion'] + "医生："
            re_or_in = f"""假设你是一名经验丰富的智能家庭助手，你的目标是根据患者和医生的对话信息，通过一系列有目的的提问来收集必要的信息，包括但不限于了解患者的症状、病史、生活方式和其他相关因素。不需要给出任何的意见，只需要进行提问，且每次仅可以提出1-2个问题，不可以提问重复的问题。下面为患者和医生的对话信息，请继续发起提问。\n\n对话信息:"""
            response = await chimpanzeestf(re_or_in,hist_ask) #继续提问
            data_recoder.append(response)
        else:
            response = await get_end(distribution, hist, hl_keywords, symptoms)
            response = response.replace('医生：','').replace('患者：','')
            data_recoder.append(response)

        # 追加写入文件
        with open('extend_answer_1.csv', 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(data_recoder)
        print(i_index)

20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42


### 访问接口

In [35]:
alll = """
用户: 我感觉胸口很疼，而且做了检查发现纵隔增宽，还查出来有纵隔囊肿。这种疼痛真的很难忍受。
医生: 有没有咳嗽的症状？
用户: 有，我有咳嗽的症状。
医生: 感觉自己咳嗽时气管有没有受压或者纵隔有没有移位？
用户: 咳嗽时感觉气管受压，也觉得纵隔有些移位。
医生: 你有没有感觉到咳嗽时有什么牵拉感？
用户: 咳嗽时感觉有些牵拉的感觉。
医生：了解您的症状了。除了这些，您有没有注意到其他的变化，比如呼吸困难或者发热？
用户：医生，我确实感觉呼吸越来越困难了，特别是平躺的时候，感觉胸口像被什么东西压着一样，必须把枕头垫得很高才能稍微好受些。最近几天还出现了低烧，体温一直在37.5度左右徘徊，而且整个人感觉特别疲惫。咳嗽时的牵拉感和胸痛也越来越明显，有时候甚至会突然一阵剧痛让我不得不停下所有动作。
医生：了解了您的症状，您在日常生活中有没有吸烟的习惯？吸烟可能会加重您的症状。
用户：医生，我平时并没有吸烟的习惯，生活里也很少接触到二手烟环境。但这次生病真的让我备受折磨，胸口疼痛、咳嗽时气管受压、纵隔移位还有牵拉感已经够难受了，现在又加上呼吸困难，平躺时胸口像压了块大石头，只能垫高枕头才能勉强喘口气，还有这持续的低烧和浑身疲惫，让我连日常活动都变得困难重重。
"""

In [36]:
input_text = "请依据医生患者的对话信息尽量给出一些大致的结果,建议的检查或者健康的生活及饮食习惯。\n对话信息:" +alll + "医生："
response = await bianque2(content=input_text) #bianque2 得出结论
response = response.replace('医生：','').replace('患者：','')
# data_recoder.append(response)
hist = "\n你是一名三甲医院的医生，请根据你和用户之间的对话并结合提供数据库，向用户提供诊断意见，诊断意见包括：可能性的疾病、疾病概述以及宜吃和忌吃食物等生活习惯。\n医患对话对话如下：\n" + alll #医患对话历史（不包含医生的结论）
result_hist = hist+ "医生：" + response
symptoms,diseases,hl_keywords = await get_ills_sym(hist,result_hist) #症状列表,疾病列表,查询关键词
extract_context = await get_illsym(diseases) #根据疾病补充症状信息
ifxunhuan,distribution,analysis = await evalu(symptoms, diseases, extract_context) #判断置信度是否大于0.5，False为是
if ifxunhuan:
    hist_ask = alll + "医生："
    re_or_in = f"""假设你是一名经验丰富的智能家庭助手，你的目标是根据患者和医生的对话信息，通过一系列有目的的提问来收集必要的信息，包括但不限于了解患者的症状、病史、生活方式和其他相关因素。不需要给出任何的意见，只需要进行提问，且每次仅可以提出1-2个问题，不可以提问重复的问题。下面为患者和医生的对话信息，请继续发起提问。\n\n对话信息:"""
    response = await chimpanzeestf(re_or_in,hist_ask) #继续提问
    print(response)
else:
    response = await get_end(distribution, hist, hl_keywords, symptoms)
    response = response.replace('医生：','').replace('患者：','')
    print(response)

<think>
好，我来分析一下用户的情况。用户提到胸口疼痛、纵隔增宽和囊肿，伴有咳嗽、气管受压、纵隔移位和牵拉感。这些症状指向可能的纵隔疾病。

首先，用户有低烧和疲惫，这提示可能存在感染或炎症反应。纵隔脓肿通常由细菌感染引起，伴随发热、疼痛加重，符合部分症状。但用户没有提到明显的波动性疼痛，所以可能性中等。

纵隔综合症可能涉及多种病因，如肿瘤、感染或其他结构异常。结合纵隔增宽和囊肿，需要考虑肿瘤的可能性。纵隔肿瘤可能导致压迫症状，如呼吸困难和平躺时的压迫感，符合用户的描述。此外，低烧可能与肿瘤相关感染有关。

炎症或感染也是可能的原因，尤其是如果囊肿并发了感染，可能会导致发热、疼痛和疲惫。但用户没有提到明显的局部红肿或其他感染迹象，因此需要进一步检查确认。

综合来看，纵隔肿瘤的可能性较高，但也不能排除脓肿或炎症的可能。建议用户到胸外科就诊，进行详细的影像学检查如CT或MRI，并考虑穿刺活检以明确诊断。
</think>

根据您的症状描述和提供的可能性疾病列表，结合知识库中的信息，我为您整理了以下诊断意见：

### 可能性分析
1. **纵隔综合症**：  
   您的症状包括胸口疼痛、纵隔移位、呼吸困难等，这些都可能与纵隔综合症相关。该病症通常由纵隔内的结构异常或功能障碍引起。

2. **纵隔脓肿**：  
   考虑到您有低烧和疲惫的症状，这可能是感染的表现，而纵隔脓肿是一种严重的感染性疾病，可能会导致类似的症状。

3. **炎症** 或 **感染**：  
   您的低烧、咳嗽以及胸痛可能与纵隔内的炎症或感染有关。这种情况需要进一步检查以明确病因。

4. **纵隔肿瘤**：  
   纵隔增宽和囊肿可能是肿瘤的表现，尤其是如果伴随呼吸困难和平躺时的压迫感，这更倾向于肿瘤的可能性。

### 饮食建议
- **宜吃食物**：  
  建议选择清淡、易消化的食物，如新鲜蔬菜（菠菜、胡萝卜）、水果（苹果、香蕉），以及高蛋白低脂肪的食物（如鱼肉、鸡肉）。这些食物有助于增强免疫力并提供必要的营养支持。

- **忌吃食物**：  
  避免辛辣、油腻和高盐分的食物，这些可能会加重您的症状。同时，减少烟酒摄入，以减轻气管受压感。

### 生活习惯建议
1. **避免吸烟及二手烟环境**：  
   您提到没有吸烟习惯，但尽量避免接触二手烟，以免加重呼吸道

In [34]:
input_text = "请依据医生患者的对话信息尽量给出一些大致的结果,建议的检查或者健康的生活及饮食习惯。\n对话信息:" +alll + "医生："
response = await bianque2(content=input_text) #bianque2 得出结论
response = response.replace('医生：','').replace('患者：','')
# data_recoder.append(response)
hist = "\n你是一名三甲医院的医生，请根据你和用户之间的对话并结合提供数据库，向用户提供诊断意见，诊断意见包括：可能性的疾病、疾病概述以及宜吃和忌吃食物等生活习惯。\n医患对话对话如下：\n" + alll #医患对话历史（不包含医生的结论）
result_hist = hist+ "医生：" + response
symptoms,diseases,hl_keywords = await get_ills_sym(hist,result_hist) #症状列表,疾病列表,查询关键词
extract_context = await get_illsym(diseases) #根据疾病补充症状信息
ifxunhuan,distribution,analysis = await evalu(symptoms, diseases, extract_context) #判断置信度是否大于0.5，False为是
# if ifxunhuan:
#     hist_ask = alll + "医生："
#     re_or_in = f"""假设你是一名经验丰富的智能家庭助手，你的目标是根据患者和医生的对话信息，通过一系列有目的的提问来收集必要的信息，包括但不限于了解患者的症状、病史、生活方式和其他相关因素。不需要给出任何的意见，只需要进行提问，且每次仅可以提出1-2个问题，不可以提问重复的问题。下面为患者和医生的对话信息，请继续发起提问。\n\n对话信息:"""
#     response = await chimpanzeestf(re_or_in,hist_ask) #继续提问
#     print(response)
# else:
response = await get_end(distribution, hist, hl_keywords, symptoms)
response = response.replace('医生：','').replace('患者：','')
print(response)

<think>
嗯，这位用户最近感觉右侧腹部疼痛，还伴随恶心、呕吐和发烧。从对话历史来看，他描述了右下腹剧烈疼痛，按压会加重，还有肿块可随呼吸移动。这些症状让我首先想到的是急性阑尾炎的可能性较大。

急性阑尾炎通常表现为右下腹痛，可能伴有发热、恶心和呕吐。用户提到的肿块位置在肚脐右下方，这与阑尾的位置相符。此外，他没有慢性疾病史，但最近工作压力大，过度劳累，这可能是诱因之一。

不过，肠梗阻也不能完全排除。肠梗阻的症状包括腹痛、胀气、排便不畅等，用户提到肚子胀、放屁少和排便不顺畅，这些都符合肠梗阻的表现。尤其是他还有口干和低烧，可能是因为脱水或感染引起的。

现在需要进一步的检查来确定病因。腹部超声可以帮助查看阑尾是否有炎症或是否存在肿块，而CT扫描则能更详细地观察肠道情况，排除肠梗阻的可能性。

治疗方面，如果是急性阑尾炎，通常建议手术切除阑尾，并配合抗生素控制感染。饮食上初期禁食，恢复后吃清淡易消化的食物。如果是肠梗阻，则可能需要调整饮食结构，避免高纤维食物，并根据严重程度决定是否需要手术治疗。

总的来说，用户的情况紧急，应该尽快就医进行详细检查和针对性治疗。
</think>

### 可能性分析

根据您描述的症状（右侧腹部疼痛、恶心呕吐、发烧、腹部肿块等），结合您的病史和当前症状，我们初步考虑以下两种可能性：

1. **急性阑尾炎**：
   - 这种疾病通常表现为右下腹痛，并可能伴有发热、恶心和呕吐。在某些情况下，患者可能会感觉到一个可触及的肿块或硬块。
   - 您的症状（如右下腹疼痛、低烧、肿块等）与急性阑尾炎的表现非常吻合。

2. **肠梗阻**：
   - 肠梗阻也可能导致类似的症状，包括腹部疼痛、恶心呕吐和发烧。但通常伴随更明显的排便障碍和腹部膨胀。
   - 您的症状中提到的右侧腹痛和肿块可能与肠梗阻有关，特别是在某些情况下（如粘连性肠梗阻），患者可能会感觉到腹部有异常包块。

### 建议

1. **就诊建议**：
   - 无论哪种可能性，您的症状都提示需要紧急就医。请立即前往医院进行进一步检查。
   - 医生可能会安排腹部超声、CT扫描等影像学检查以明确诊断。
   - 如果确诊为急性阑尾炎，通常会推荐手术治疗（阑尾切除术）。

2. **饮食建议**：
   - 在就医前，请避免进食固体食物。如果您感到口渴，可以尝试少量饮用